# 🔍 End-to-End RAG Pipeline

> **Educational Notebook 02**: Complete RAG flow from document to answer.

---

## 📋 What We'll Cover

1. Text Extraction from PDF/DOCX
2. Token-Aware Chunking
3. Embedding Generation
4. Vector Storage (Qdrant)
5. Retrieval
6. Prompt Building
7. Answer Generation

In [ ]:
# Setup imports from src/
import sys
sys.path.insert(0, '..')

from src.core.config import settings
from src.domain.entities import Chunk, TenantId, DocumentId, ChunkSpec, Answer
from src.application.services.chunking import chunk_text_token_aware
from src.application.services.prompt_builder import build_rag_prompt, build_chat_messages
from src.application.services.fusion import rrf_fusion
from src.application.services.scoring import ScoredChunk

## 📝 Step 1: Text Extraction

Extract text from documents using pypdf and python-docx.

In [ ]:
# Simulate extracted text (in production, use DefaultTextExtractor)
sample_document = """
# Machine Learning Fundamentals

Machine learning (ML) is a subset of artificial intelligence that enables computers
to learn from data without being explicitly programmed. There are three main types:

## Supervised Learning
In supervised learning, the algorithm learns from labeled training data. Examples include:
- Classification: Predicting categories (spam detection, image classification)
- Regression: Predicting continuous values (house prices, stock prices)

## Unsupervised Learning  
Unsupervised learning finds patterns in unlabeled data:
- Clustering: Grouping similar items (customer segmentation)
- Dimensionality reduction: Reducing features while preserving information (PCA)

## Reinforcement Learning
An agent learns by interacting with an environment, receiving rewards or penalties.
Applications include game playing (AlphaGo) and robotics.

## Deep Learning
Deep learning uses neural networks with many layers. Key architectures:
- CNNs: Convolutional Neural Networks for images
- RNNs: Recurrent Neural Networks for sequences
- Transformers: Attention-based models for language (BERT, GPT)
"""

print(f"Document length: {len(sample_document)} characters")

## ✂️ Step 2: Chunking

Split text into overlapping chunks for retrieval.

In [ ]:
# Chunk the document
chunks_text = chunk_text_token_aware(
    sample_document,
    spec=ChunkSpec(max_tokens=150, overlap_tokens=30)
)

print(f"Created {len(chunks_text)} chunks:")
for i, text in enumerate(chunks_text, 1):
    print(f"\n--- Chunk {i} ({len(text)} chars) ---")
    print(text[:200] + "..." if len(text) > 200 else text)

## 🔢 Step 3: Create Chunk Objects

In [ ]:
tenant = TenantId("demo_user")
doc_id = DocumentId("ml_fundamentals")

chunks = [
    Chunk(
        id=f"chunk_{i}",
        tenant_id=tenant,
        document_id=doc_id,
        text=text
    )
    for i, text in enumerate(chunks_text, 1)
]

print(f"Created {len(chunks)} Chunk objects")

## 🔍 Step 4: Simulate Retrieval

In production, this would query Qdrant for vector similarity.

In [ ]:
# Simulate retrieval - select chunks containing relevant keywords
question = "What is the difference between supervised and unsupervised learning?"

# Simple keyword matching (in production: vector + keyword search)
relevant_chunks = [
    c for c in chunks
    if "supervised" in c.text.lower() or "unsupervised" in c.text.lower()
]

print(f"Found {len(relevant_chunks)} relevant chunks:")
for c in relevant_chunks:
    print(f"  - {c.id}: {c.text[:80]}...")

## 📜 Step 5: Build Prompt

In [ ]:
# Build RAG prompt with guardrails
prompt = build_rag_prompt(
    question=question,
    chunks=relevant_chunks,
    max_context_chars=4000
)

print("=" * 60)
print("GENERATED PROMPT:")
print("=" * 60)
print(prompt)

## 💬 Step 6: Generate Answer (Simulated)

In production, this would call OpenAI or Ollama.

In [ ]:
# Simulate LLM response
simulated_answer = """
Based on the context provided:

**Supervised Learning** uses labeled training data where the algorithm learns
from examples with known outcomes. Common applications include classification
(like spam detection) and regression (like predicting house prices).

**Unsupervised Learning** works with unlabeled data and finds patterns
without predefined categories. It's used for clustering (grouping similar items)
and dimensionality reduction (simplifying data while preserving information).

The key difference is that supervised learning requires labeled examples,
while unsupervised learning discovers structure in unlabeled data.
"""

# Create Answer object
answer = Answer(
    text=simulated_answer,
    sources=[c.id for c in relevant_chunks]
)

print("=" * 60)
print("ANSWER:")
print("=" * 60)
print(answer.text)
print(f"\nSources: {list(answer.sources)}")

## 📚 Next Steps

Continue with:
- **03_hybrid_search_and_rerank.ipynb** - Deep dive into hybrid retrieval and RRF fusion